## Scan for Yaml files:

In [ ]:
from pathlib import Path

benefits_folder = Path('./aides')

def extract_benefits_paths(benefits_folder):
    isYAMLfile = lambda path : str(path).endswith('.yml') or str(path).endswith('.yaml')
    liste_fichiers = [str(benefit) for benefit in benefits_folder.iterdir() if isYAMLfile(benefit)]
    return liste_fichiers

benefits_files_paths = extract_benefits_paths(benefits_folder)
benefits_files_paths

## Open and parse files: make a list of Dict

In [ ]:
import yaml

def extract_benefit_file_content(aide_path):
    return yaml.safe_load(open(aide_path))

def extract_benefits_from_yamls(path):
    benefits_files_paths = extract_benefits_paths(path)
    benefits = [extract_benefit_file_content(path) for path in benefits_files_paths]
    return benefits

In [ ]:
benefits = extract_benefits_from_yamls(benefits_folder)

##  Make a list of [profils] [type] in all aides given

In [ ]:
def get_profils_names_from_aides_list(path : str):
    
    get_aide_profiles_names = lambda aide: [profile['type'] for profile in aide['profils']]
    
    return [profile for benefit in benefits for profile in get_aide_profiles_names(benefit)]

profiles_names = get_profils_names_from_aides_list(benefits)
profiles_names

# Make a list with all conditons:
	- conditions generales
	- conditions in [profils][type]

In [ ]:
def extract_conditions(benefit):
    
    conditions = [condition['type'] for condition in benefit['conditions_generales']]
    profil_conditions = [condition['type'] for profil in benefit['profils'] if 'conditions' in profil for condition in profil['conditions'] ]
    
    return conditions + profil_conditions




def get_all_conditions_from_benefits(path: str):
    
    return [condition for benefit in benefits for condition in extract_conditions(benefit)]


conditions_names = get_all_conditions_from_benefits(benefits)
conditions_names

# Count frequence of profils and conditions appearances

In [ ]:
def count_list_item_frequence(list : list):
    elements_counter = {}
    for element in list:
        if element in elements_counter:
            elements_counter[element] +=1
        else:
            elements_counter[element] = 1
    return (sorted(elements_counter.items(), key=lambda x:x[1]))

profil_names_frequence = count_list_item_frequence(profiles_names)
condition_frequence = count_list_item_frequence(conditions_names)

In [ ]:
condition_frequence

In [ ]:
profil_names_frequence